# Google Cloud Setup Procedure

**Compute Instance Details**

OS: Ubuntu 16.04 LTS HVM. Can be found in the Marketplace at zero cost.

Size: Absolute minimum of t2.small - some of the gem dependencies won't compile on a micro or nano due to RAM limitations!

Security Group: any

**DB Instance Details**

DB: MariaDB

Size: t2.micro (Might need expanding for production)

**Dependencies**

gpg --keyserver hkp://keys.gnupg.net --recv-keys 409B6B1796C275462A1703113804BB82D39DC0E3

\curl -sSL https://get.rvm.io | bash -s stable

rvm install 2.1.1

rvm use ruby-2.1.1

rvm gemset create lot_structure

rvm use ruby-2.1.1@lot_structure

gem install bundler

rvm autolibs packages

sudo apt-get install libmysqlclient-dev graphicsmagick-libmagick-dev-compat imagemagick libxslt1-dev libxml2-dev libmagickcore-dev libmagickwand-dev

sudo ln -s /usr/lib/x86_64-linux-gnu/ImageMagick-6.8.9/bin-Q16/Magick-config /usr/bin/Magick-config

**Database.yml**

**Reverse Proxy**

#user  nobody;
worker_processes  1;

#error_log  logs/error.log;
#error_log  logs/error.log  notice;
#error_log  logs/error.log  info;

#pid        logs/nginx.pid;
pid         /var/run/nginx.pid;

events {
    worker_connections  1024;
}

http {
    passenger_root /home/deploy/.rvm/gems/ruby-2.1.1@category_dojo/gems/passenger-4.0.45;
    passenger_ruby /usr/local/rvm/gems/ruby-2.1.1@lot_structure/wrappers/ruby;  
    # passenger_max_pool_size 25; # NM 2015-04-20 increased memory from 8 to 16GB
    # passenger_max_pool_size 45; # NM 2016-05-11 server outage due to too many processes
    passenger_max_pool_size 35;
    include       mime.types;
    default_type  application/octet-stream;

    #log_format  main  '$remote_addr - $remote_user [$time_local] "$request" '
    #                  '$status $body_bytes_sent "$http_referer" '
    #                  '"$http_user_agent" "$http_x_forwarded_for"';

    #access_log  logs/access.log  main;

    sendfile        on;
    #tcp_nopush     on;

    #keepalive_timeout  0;
    keepalive_timeout  65;

    # NM 2013-12-06 - add gzip settings based on http://wiki.linuxwall.info/doku.php/en:ressources:dossiers:nginx:nginx_performance_tuning#nginx_performance_tuning
    gzip  on;
    gzip_comp_level 9;
    gzip_min_length 1400;
    gzip_types  text/plain text/css image/png image/gif image/jpeg application/x-javascript text/xml application/xml application/xml+rss text/javascript;
    gzip_vary  on;
    gzip_http_version 1.1;
    gzip_disable "MSIE [1-6]\.(?!.*SV1)";    
    # NM 2014-10-22 - POODLE vulnerability, disable SSL.
    ssl_protocols  TLSv1 TLSv1.1 TLSv1.2;  # don’t use SSLv3 ref: POODLE

    # NM 2015-09-02 need to upload larger files
    client_max_body_size 100M;
    server {
       listen 80;
       server_name localhost;
       root /var/www/mdapps/production/current/public;   # <--- be sure to point to 'public'!
       passenger_enabled on;

       location ~ ^/assets/ {
         gzip_static on; # to serve pre-gzipped version
         expires 1y;
         add_header Cache-Control public;
         add_header ETag "";
         # Fonts need to allow CORS for Firefox...
         # http://stackoverflow.com/questions/15080434/how-to-display-font-awesome-icons-in-firefox
         if ($request_filename ~* ^.*?\.(eot)|(ttf)|(woff)|(svg)|(otf)$) {
           add_header Access-Control-Allow-Origin *;
         }
         break;
       }

        location = /nginx_stub_status {
            stub_status on;
            allow 127.0.0.1;
            deny all;
        }
    }

    # another virtual host using mix of IP-, name-, and port-based configuration
    #
    #server {
    #    listen       8000;
    #    listen       somename:8080;
    #    server_name  somename  alias  another.alias;

    #    location / {
    #        root   html;
    #        index  index.html index.htm;
    #    }
    #}

    map $http_upgrade $connection_upgrade {
        default upgrade;
        '' close;
    }

    upstream websocket {
       server localhost:9292;
    } 

    # HTTPS server
    server {
       listen 443 ssl;
       server_name 80.193.71.165;
       root /var/www/mdapps/production/current/public;   # <--- be sure to point to 'public'!
       passenger_enabled on;

       # REMEMBER - also need to update marketdojo-pubsub-server 
       #ssl_certificate      /etc/httpd/conf/ssl.crt/marketdojo-ssl-bundle-2015.crt;
       #ssl_certificate_key  /etc/httpd/conf/ssl.crt/marketdojo_2015.key;
       ssl_certificate      /etc/httpd/conf/ssl.crt/marketdojo2016ssl.crt;
       ssl_certificate_key  /etc/httpd/conf/ssl.crt/marketdojo_2016.key;

       ssl_session_cache    shared:SSL:1m;
       ssl_session_timeout  5m;


       # https://weakdh.org/sysadmin.html
       ssl_ciphers 'ECDHE-RSA-AES128-GCM-SHA256:ECDHE-ECDSA-AES128-GCM-SHA256:ECDHE-RSA-AES256-GCM-SHA384:ECDHE-ECDSA-AES256-GCM-SHA384:DHE-RSA-AES128-GCM-SHA256:DHE-DSS-AES128-GCM-SHA256:kEDH+AESGCM:ECDHE-RSA-AES128-SHA256:ECDHE-ECDSA-AES128-SHA256:ECDHE-RSA-AES128-SHA:ECDHE-ECDSA-AES128-SHA:ECDHE-RSA-AES256-SHA384:ECDHE-ECDSA-AES256-SHA384:ECDHE-RSA-AES256-SHA:ECDHE-ECDSA-AES256-SHA:DHE-RSA-AES128-SHA256:DHE-RSA-AES128-SHA:DHE-DSS-AES128-SHA256:DHE-RSA-AES256-SHA256:DHE-DSS-AES256-SHA:DHE-RSA-AES256-SHA:AES128-GCM-SHA256:AES256-GCM-SHA384:AES128-SHA256:AES256-SHA256:AES128-SHA:AES256-SHA:AES:CAMELLIA:DES-CBC3-SHA:!aNULL:!eNULL:!EXPORT:!DES:!RC4:!MD5:!PSK:!aECDH:!EDH-DSS-DES-CBC3-SHA:!EDH-RSA-DES-CBC3-SHA:!KRB5-DES-CBC3-SHA';
       ssl_dhparam /etc/httpd/conf/ssl.crt/dhparams.pem;
       ssl_prefer_server_ciphers  on;

      add_header Strict-Transport-Security "max-age=631138519";

       location ~ ^/assets/ {
         gzip_static on; # to serve pre-gzipped version
         expires 1y;
         add_header Cache-Control public;
         add_header ETag "";
         # Fonts need to allow CORS for Firefox...
         # http://stackoverflow.com/questions/15080434/how-to-display-font-awesome-icons-in-firefox
         if ($request_filename ~* ^.*?\.(eot)|(ttf)|(woff)|(svg)|(otf)$) {
           add_header Access-Control-Allow-Origin *;
         }
         break;
       } 

       location /socket.io/ {
         proxy_pass https://websocket;
         proxy_http_version 1.1;
         proxy_set_header Upgrade $http_upgrade;
         proxy_set_header Connection "Upgrade";
       }
    }

    # HTTPS server for Aggate
    server {
       listen 443 ssl;
       server_name .aggate.com;
       root /var/www/mdapps/production/current/public;   # <--- be sure to point to 'public'!
       passenger_enabled on;

       ssl_certificate      /etc/httpd/conf/ssl.crt/aggate.com.crt;
       ssl_certificate_key  /etc/httpd/conf/ssl.crt/aggate-md.key;

       ssl_session_cache    shared:SSL:1m;
       ssl_session_timeout  5m;

       # https://weakdh.org/sysadmin.html
       ssl_ciphers 'ECDHE-RSA-AES128-GCM-SHA256:ECDHE-ECDSA-AES128-GCM-SHA256:ECDHE-RSA-AES256-GCM-SHA384:ECDHE-ECDSA-AES256-GCM-SHA384:DHE-RSA-AES128-GCM-SHA256:DHE-DSS-AES128-GCM-SHA256:kEDH+AESGCM:ECDHE-RSA-AES128-SHA256:ECDHE-ECDSA-AES128-SHA256:ECDHE-RSA-AES128-SHA:ECDHE-ECDSA-AES128-SHA:ECDHE-RSA-AES256-SHA384:ECDHE-ECDSA-AES256-SHA384:ECDHE-RSA-AES256-SHA:ECDHE-ECDSA-AES256-SHA:DHE-RSA-AES128-SHA256:DHE-RSA-AES128-SHA:DHE-DSS-AES128-SHA256:DHE-RSA-AES256-SHA256:DHE-DSS-AES256-SHA:DHE-RSA-AES256-SHA:AES128-GCM-SHA256:AES256-GCM-SHA384:AES128-SHA256:AES256-SHA256:AES128-SHA:AES256-SHA:AES:CAMELLIA:DES-CBC3-SHA:!aNULL:!eNULL:!EXPORT:!DES:!RC4:!MD5:!PSK:!aECDH:!EDH-DSS-DES-CBC3-SHA:!EDH-RSA-DES-CBC3-SHA:!KRB5-DES-CBC3-SHA';
       ssl_dhparam /etc/httpd/conf/ssl.crt/dhparams.pem;
       ssl_prefer_server_ciphers  on;

      add_header Strict-Transport-Security "max-age=631138519";

       location ~ ^/assets/ {
         gzip_static on; # to serve pre-gzipped version
         expires 1y;
         add_header Cache-Control public;
         add_header ETag "";
         # Fonts need to allow CORS for Firefox...
         # http://stackoverflow.com/questions/15080434/how-to-display-font-awesome-icons-in-firefox
         if ($request_filename ~* ^.*?\.(eot)|(ttf)|(woff)|(svg)|(otf)$) {
           add_header Access-Control-Allow-Origin *;
         }
         break;
       }

       location /socket.io/ {
         proxy_pass https://websocket;
         proxy_http_version 1.1;
         proxy_set_header Upgrade $http_upgrade;
         proxy_set_header Connection "Upgrade";
       }
    }

    # HTTPS server for RateClix
    server {
       listen 443 ssl;
       server_name .rateclix.com;
       root /var/www/mdapps/production/current/public;   # <--- be sure to point to 'public'!
       passenger_enabled on;

       ssl_certificate      /etc/httpd/conf/ssl.crt/rateclix_bundle.crt;
       ssl_certificate_key  /etc/httpd/conf/ssl.crt/rateclix.key;

       ssl_session_cache    shared:SSL:1m;
       ssl_session_timeout  5m;


       # https://weakdh.org/sysadmin.html
       ssl_ciphers 'ECDHE-RSA-AES128-GCM-SHA256:ECDHE-ECDSA-AES128-GCM-SHA256:ECDHE-RSA-AES256-GCM-SHA384:ECDHE-ECDSA-AES256-GCM-SHA384:DHE-RSA-AES128-GCM-SHA256:DHE-DSS-AES128-GCM-SHA256:kEDH+AESGCM:ECDHE-RSA-AES128-SHA256:ECDHE-ECDSA-AES128-SHA256:ECDHE-RSA-AES128-SHA:ECDHE-ECDSA-AES128-SHA:ECDHE-RSA-AES256-SHA384:ECDHE-ECDSA-AES256-SHA384:ECDHE-RSA-AES256-SHA:ECDHE-ECDSA-AES256-SHA:DHE-RSA-AES128-SHA256:DHE-RSA-AES128-SHA:DHE-DSS-AES128-SHA256:DHE-RSA-AES256-SHA256:DHE-DSS-AES256-SHA:DHE-RSA-AES256-SHA:AES128-GCM-SHA256:AES256-GCM-SHA384:AES128-SHA256:AES256-SHA256:AES128-SHA:AES256-SHA:AES:CAMELLIA:DES-CBC3-SHA:!aNULL:!eNULL:!EXPORT:!DES:!RC4:!MD5:!PSK:!aECDH:!EDH-DSS-DES-CBC3-SHA:!EDH-RSA-DES-CBC3-SHA:!KRB5-DES-CBC3-SHA';
       ssl_dhparam /etc/httpd/conf/ssl.crt/dhparams.pem;
       ssl_prefer_server_ciphers  on;

      add_header Strict-Transport-Security "max-age=631138519";

       location ~ ^/assets/ {
         gzip_static on; # to serve pre-gzipped version
         expires 1y;
         add_header Cache-Control public;
         add_header ETag "";
         # Fonts need to allow CORS for Firefox...
         # http://stackoverflow.com/questions/15080434/how-to-display-font-awesome-icons-in-firefox
         if ($request_filename ~* ^.*?\.(eot)|(ttf)|(woff)|(svg)|(otf)$) {
           add_header Access-Control-Allow-Origin *;
         }
         break;
       }

       location /socket.io/ {
         proxy_pass https://websocket;
         proxy_http_version 1.1;
         proxy_set_header Upgrade $http_upgrade;
         proxy_set_header Connection "Upgrade";
       }
    }

    # HTTPS server for Savers
    server {
       listen 443 ssl;
       server_name .savers.com;
       root /var/www/mdapps/production/current/public;   # <--- be sure to point to 'public'!
       passenger_enabled on;

       ssl_certificate      /etc/httpd/conf/ssl.crt/esourcing.savers.com.crt;
       ssl_certificate_key  /etc/httpd/conf/ssl.crt/esourcing.savers.com.rsa;

       ssl_session_cache    shared:SSL:1m;
       ssl_session_timeout  5m;


       # https://weakdh.org/sysadmin.html
       ssl_ciphers 'ECDHE-RSA-AES128-GCM-SHA256:ECDHE-ECDSA-AES128-GCM-SHA256:ECDHE-RSA-AES256-GCM-SHA384:ECDHE-ECDSA-AES256-GCM-SHA384:DHE-RSA-AES128-GCM-SHA256:DHE-DSS-AES128-GCM-SHA256:kEDH+AESGCM:ECDHE-RSA-AES128-SHA256:ECDHE-ECDSA-AES128-SHA256:ECDHE-RSA-AES128-SHA:ECDHE-ECDSA-AES128-SHA:ECDHE-RSA-AES256-SHA384:ECDHE-ECDSA-AES256-SHA384:ECDHE-RSA-AES256-SHA:ECDHE-ECDSA-AES256-SHA:DHE-RSA-AES128-SHA256:DHE-RSA-AES128-SHA:DHE-DSS-AES128-SHA256:DHE-RSA-AES256-SHA256:DHE-DSS-AES256-SHA:DHE-RSA-AES256-SHA:AES128-GCM-SHA256:AES256-GCM-SHA384:AES128-SHA256:AES256-SHA256:AES128-SHA:AES256-SHA:AES:CAMELLIA:DES-CBC3-SHA:!aNULL:!eNULL:!EXPORT:!DES:!RC4:!MD5:!PSK:!aECDH:!EDH-DSS-DES-CBC3-SHA:!EDH-RSA-DES-CBC3-SHA:!KRB5-DES-CBC3-SHA';
       ssl_dhparam /etc/httpd/conf/ssl.crt/dhparams.pem;
       ssl_prefer_server_ciphers  on;

      add_header Strict-Transport-Security "max-age=631138519";

       location ~ ^/assets/ {
         gzip_static on; # to serve pre-gzipped version
         expires 1y;
         add_header Cache-Control public;
         add_header ETag "";
         # Fonts need to allow CORS for Firefox...
         # http://stackoverflow.com/questions/15080434/how-to-display-font-awesome-icons-in-firefox
         if ($request_filename ~* ^.*?\.(eot)|(ttf)|(woff)|(svg)|(otf)$) {
           add_header Access-Control-Allow-Origin *;
         }
         break;
       }

       location /socket.io/ {
         proxy_pass https://websocket;
         proxy_http_version 1.1;
         proxy_set_header Upgrade $http_upgrade;
         proxy_set_header Connection "Upgrade";
       }
    }
}